In [27]:
import pandas as pd
from tqdm import tqdm
from math import sin, cos, sqrt, atan2, radians

In [21]:
bus_stop_data = pd.read_csv('data/bus_stops.csv')
mrt_station_data = pd.read_csv('data/mrt_stations_with_geo_data.csv').drop_duplicates()

In [22]:
#function to calculate distance given lat and long of two points and return the distance in metres
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * 1000

    return distance


In [37]:
distance = []
station_name = []
station_lat = []
station_lon = []
for i,row in tqdm(bus_stop_data.iterrows()):
    # Get nearest MRT station using mr_station_data
    min_distance = 1000000
    nearest_station = None
    lat_nearest_station = None
    lon_nearest_station = None
    for j,station_row in mrt_station_data.iterrows():
        dist = calculate_distance(row['Latitude'], row['Longitude'], station_row['latitude'], station_row['longitude'])
        if dist < min_distance:
            min_distance = dist
            nearest_station = station_row['station_name']    
            lat_nearest_station = station_row['latitude']
            lon_nearest_station = station_row['longitude']  
    
    distance.append(min_distance)
    station_name.append(nearest_station)
    station_lat.append(lat_nearest_station)
    station_lon.append(lon_nearest_station)

5137it [00:22, 224.32it/s]


In [41]:
new_data  = bus_stop_data.\
    rename(columns={'Latitude':'bus_latitude','Longitude':'bus_longitude'}).\
    assign(nearest_station_name = station_name, 
           nearest_station_lat = station_lat, 
           nearest_station_lon = station_lon, 
           distance_to_nearest_station = distance)

In [44]:
new_data.to_csv('processed_data/bus_stops_with_nearest_mrt.csv', index=False)